# Manipulação de Strings no Apache Spark

O Apache Spark fornece uma variedade de funções para manipular strings em DataFrames. Essas funções são úteis para realizar tarefas como limpeza, formatação, extração e transformação de dados de texto.

## Funções de Manipulação de Strings

As funções a seguir são comumente usadas para manipulação de strings:

| Função                       | Descrição                                                    | Exemplo de Caso de Uso                  |
|------------------------------|--------------------------------------------------------------|---------------------------------------|
| `concat(str1, str2, ...)`    | Concatena strings juntas                                     | Unir nomes em um único campo.          |
| `substring(str, pos, len)`   | Extrai uma parte da string a partir de uma posição e comprimento específicos | Obter prefixos, sufixos ou substrings. |
| `length(str)`                | Retorna o comprimento da string                              | Verificar a validade de identificadores. |
| `trim(str)`                  | Remove espaços em branco do início e do final da string     | Limpeza de dados de texto.             |
| `lower(str)`                 | Converte todos os caracteres para minúsculas                | Padronização de maiúsculas e minúsculas. |
| `upper(str)`                 | Converte todos os caracteres para maiúsculas                | Padronização de maiúsculas e minúsculas. |
| `initcap(str)`               | Converte a primeira letra de cada palavra para maiúscula    | Formatação de nomes.                   |
| `replace(str, target, replacement)` | Substitui uma sequência por outra                    | Correção de erros de digitação.         |
| `regexp_replace(str, pattern, replacement)` | Substitui padrões regulares por outra sequência | Extração e correção de informações.    |
| `split(str, pattern)`        | Divide a string com base em um padrão                       | Separar valores em uma lista.           |
| `instr(str, substr)`         | Encontra a posição da primeira ocorrência de uma subtring  | Validação de presença de informações.  |
| `substring_index(str, delimiter, count)` | Divide a string com base em um delimitador e recupera a subtring em uma posição específica | Separar e recuperar informações.  |


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Exemplo").config("spark.jars.packages", "org.postgresql:postgresql:42.2.24").getOrCreate()

In [2]:
df = (
    spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("encoding","utf-8")
    .option("delimiter",";") #ou sep
    .option("header", "true")
    .load("data/landing/air_cia/")
)
df.printSchema()

root
 |-- Razão Social: string (nullable = true)
 |-- ICAO IATA: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Atividades Aéreas: string (nullable = true)
 |-- Endereço Sede: string (nullable = true)
 |-- Telefone: string (nullable = true)
 |-- E-Mail: string (nullable = true)
 |-- Decisão Operacional: string (nullable = true)
 |-- Data Decisão Operacional: string (nullable = true)
 |-- Validade Operacional: string (nullable = true)



In [3]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------
 Razão Social             | ABSA - AEROLINHAS BRASILEIRAS S.A.                                                                                           
 ICAO IATA                | LTG M3                                                                                                                       
 CNPJ                     | 00.074.635/0001-33                                                                                                           
 Atividades Aéreas        | TRANSPORTE AÉREO REGULAR                                                                                                     
 Endereço Sede            | AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA SANTOS DUMONT, KM 66, SISTEMA VIÁRIO PRINCIPAL, S/ Nº, 13.052-970, CAMPINAS-SP 
 Telefone                 | (11) 5582-8055                                  

In [4]:
df = df.toDF(*[col.lower().replace(" ","_") for col in df.columns])
df.printSchema()

root
 |-- razão_social: string (nullable = true)
 |-- icao_iata: string (nullable = true)
 |-- cnpj: string (nullable = true)
 |-- atividades_aéreas: string (nullable = true)
 |-- endereço_sede: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- e-mail: string (nullable = true)
 |-- decisão_operacional: string (nullable = true)
 |-- data_decisão_operacional: string (nullable = true)
 |-- validade_operacional: string (nullable = true)



In [5]:
import pyspark.sql.functions as F

# A expressão regular [\./-] significa que qualquer ponto '.', barra '/' ou hífen '-' será substituído
df = (
    df
    .withColumn("cnpj", F.regexp_replace("cnpj","[\./-]", ""))
    .withColumn("telefone", F.regexp_replace("telefone","[^0-9]", ""))
    .withColumn("icao",F.split("icao_iata"," ")[0])
    .withColumn("iata",F.split("icao_iata"," ")[1])
    .withColumn("atividades_aéreas",F.upper("atividades_aéreas"))               
)

In [6]:
(
    df
    .write
    .format("parquet")
    .mode("overwrite")
    .save("/home/app/data/1.bronze/air_cia/")
)